In [1]:
import sys
sys.path.append('./FuxiCTR-main/')
import fuxictr
fuxictr.__version__

'1.2.3'

In [11]:
import os
import logging
from datetime import datetime
from fuxictr import datasets
from fuxictr.datasets.taobao import FeatureEncoder
from fuxictr.features import FeatureMap
from fuxictr.utils import load_config, set_logger, print_to_json
from fuxictr.pytorch.models import DeepFM
from fuxictr.pytorch.torch_utils import seed_everything
import pandas as pd
from sklearn.model_selection import train_test_split

In [7]:
#df_reviews = pd.read_json("../yelp_dataset/yelp_academic_dataset_review.json", lines=True)
df_reviews = pd.read_csv("s3://w210-trip/data/review_philly.csv")

In [5]:
df_reviews.head(3)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30


In [8]:
def create_target(row):
    if row["stars"] <= 3:
        return 0
    else:
        return 1

In [9]:
df_reviews["target"] = df_reviews.apply(lambda x: create_target(x), axis=1)

In [10]:
df_reviews.head(3)

,_id,review_id,user_id,business_id,stars,useful,funny,cool,text,date,compliment_count,target
0,631e9f7fedf65856ab0dfe3b,JrIxlS1TzJ-iCu79ul40cQ,eUta8W_HdHMXPzLBBZhL1A,04UD14gamNjLY0IDYVhHJg,1.0,1.0,2.0,1.0,I am a long term frequent customer of this est...,2015-09-23 23:10:31,NaN,0
1,631e9f7fedf65856ab0dfe3e,8JFGBuHMoiNDyfcxuWNtrA,smOvOajNG0lS4Pq7d8g4JQ,RZtGWDLCAtuipwaZ-UfjmQ,4.0,0.0,0.0,0.0,Good food--loved the gnocchi with marinara\nth...,2009-10-14 19:57:14,NaN,1
2,631e9f7fedf65856ab0dfe42,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1.0,0.0,1.0,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03,NaN,1


In [14]:
df_train, df_val = train_test_split(
    df_reviews, 
    test_size=0.2, 
    shuffle=True, 
    random_state=3
)

In [20]:
df_train.to_csv("./data_sample/train_sample.csv", index=False)
df_val.to_csv("./data_sample/valid_sample.csv", index=False)

In [21]:
feature_cols = [{'name': ["user_id", "business_id"],
                 'active': True, 'dtype': 'str', 'type': 'categorical'}]
label_col = {'name': 'target', 'dtype': float}

params = {'model_id': 'DeepFM_trial',
          'dataset_id': 'yelp_tiny',
          'train_data': './data_sample/train_sample.csv',
          'valid_data': './data_sample/valid_sample.csv',
          # 'test_data': 'FuxiCTR-main/data/tiny_data/test_sample.csv',
          'model_root': './checkpoints/',
          'data_root': './data_sample',
          'feature_cols': feature_cols,
          'label_col': label_col,
          'embedding_regularizer': 0,
          'net_regularizer': 0,
          'hidden_units': [64, 64],
          'hidden_activations': "relu",
          'learning_rate': 1e-3,
          'net_dropout': 0,
          'batch_norm': False,
          'optimizer': 'adam',
          'task': 'binary_classification',
          'loss': 'binary_crossentropy',
          'metrics': ['logloss', 'AUC'],
          'min_categr_count': 1,
          'embedding_dim': 10,
          'batch_size': 16,
          'epochs': 3,
          'shuffle': True,
          'seed': 2019,
          'monitor': 'AUC',
          'monitor_mode': 'max',
          'use_hdf5': True,
          'pickle_feature_encoder': True,
          'save_best_only': True,
          'every_x_epochs': 1,
          'patience': 2,
          'num_workers': 1,
          'partition_block_size': -1,
          'verbose': 1,
          'version': 'pytorch',
          'gpu': -1}

# Set the logger and random seed
set_logger(params)
logging.info(print_to_json(params))
seed_everything(seed=params['seed'])

2022-10-15 14:50:31,591 P56629 INFO {
    "batch_norm": "False",
    "batch_size": "16",
    "data_root": "./data_sample",
    "dataset_id": "yelp_tiny",
    "embedding_dim": "10",
    "embedding_regularizer": "0",
    "epochs": "3",
    "every_x_epochs": "1",
    "feature_cols": "[{'name': ['user_id', 'business_id'], 'active': True, 'dtype': 'str', 'type': 'categorical'}]",
    "gpu": "-1",
    "hidden_activations": "relu",
    "hidden_units": "[64, 64]",
    "label_col": "{'name': 'target', 'dtype': <class 'float'>}",
    "learning_rate": "0.001",
    "loss": "binary_crossentropy",
    "metrics": "['logloss', 'AUC']",
    "min_categr_count": "1",
    "model_id": "DeepFM_trial",
    "model_root": "./checkpoints/",
    "monitor": "AUC",
    "monitor_mode": "max",
    "net_dropout": "0",
    "net_regularizer": "0",
    "num_workers": "1",
    "optimizer": "adam",
    "partition_block_size": "-1",
    "patience": "2",
    "pickle_feature_encoder": "True",
    "save_best_only": "True",
  

In [22]:
# Set feature_encoder that defines how to preprocess data
feature_encoder = FeatureEncoder(feature_cols, 
                                 label_col, 
                                 dataset_id=params['dataset_id'], 
                                 data_root=params["data_root"])

# Build dataset from csv to h5
datasets.build_dataset(feature_encoder, 
                       train_data=params["train_data"], 
                       valid_data=params["valid_data"],)
                       #test_data=params["test_data"])

2022-10-15 14:50:33,798 P56629 INFO Set up feature encoder...
2022-10-15 14:50:33,799 P56629 INFO Reading file: ./data_sample/train_sample.csv
2022-10-15 14:50:37,983 P56629 INFO Reading file: ./data_sample/valid_sample.csv
2022-10-15 14:50:39,029 P56629 INFO Preprocess feature columns...
2022-10-15 14:50:39,177 P56629 INFO Fit feature encoder...
2022-10-15 14:50:39,178 P56629 INFO Processing column: {'name': 'user_id', 'active': True, 'dtype': 'str', 'type': 'categorical'}
2022-10-15 14:50:39,731 P56629 INFO Processing column: {'name': 'business_id', 'active': True, 'dtype': 'str', 'type': 'categorical'}
2022-10-15 14:50:39,807 P56629 INFO Set feature index...
2022-10-15 14:50:39,808 P56629 INFO Pickle feature_encoder: ./data_sample/yelp_tiny/feature_encoder.pkl
2022-10-15 14:50:39,858 P56629 INFO Save feature_map to json: ./data_sample/yelp_tiny/feature_map.json
2022-10-15 14:50:39,860 P56629 INFO Set feature encoder done.
2022-10-15 14:50:39,860 P56629 INFO Transform feature columns

In [25]:
# Get feature_map that defines feature specs
feature_map = feature_encoder.feature_map

# Get train and validation data generator from h5
data_dir = os.path.join(params['data_root'], params['dataset_id'])
train_gen, valid_gen = datasets.h5_generator(feature_map, 
                                             stage='train', 
                                             train_data=os.path.join(data_dir, 'train.h5'),
                                             valid_data=os.path.join(data_dir, 'valid.h5'),
                                             batch_size=params['batch_size'],
                                             shuffle=params['shuffle'])

2022-10-15 14:51:19,380 P56629 INFO Loading data...
2022-10-15 14:51:19,389 P56629 INFO Loading data from h5: ./data_sample/yelp_tiny/train.h5
2022-10-15 14:51:19,395 P56629 INFO Loading data from h5: ./data_sample/yelp_tiny/valid.h5
2022-10-15 14:51:19,400 P56629 INFO Train samples: total/684064, pos/472869, neg/211195, ratio/69.13%, blocks/1
2022-10-15 14:51:19,400 P56629 INFO Validation samples: total/171016, pos/118244, neg/52772, ratio/69.14%, blocks/1
2022-10-15 14:51:19,401 P56629 INFO Loading train data done.


In [26]:
model = DeepFM(feature_map, **params)
model.count_parameters() # print number of parameters used in model
model.fit_generator(train_gen, 
                    validation_data=valid_gen, 
                    epochs=params['epochs'],
                    verbose=params['verbose'])

2022-10-15 14:51:20,184 P56629 INFO Total number of parameters: 2482142.
2022-10-15 14:51:20,184 P56629 INFO Start training: 42754 batches/epoch
2022-10-15 14:51:20,185 P56629 INFO ************ Epoch=1 start ************


100%|████████████████████████████████████████████████████████████████████████████| 10689/10689 [00:11<00:00, 917.51it/s]

2022-10-15 15:04:01,146 P56629 INFO [Metrics] logloss: 0.536154 - AUC: 0.739063
2022-10-15 15:04:01,147 P56629 INFO Save best model: monitor(max): 0.739063
2022-10-15 15:04:01,162 P56629 INFO --- 42754/42754 batches finished ---



100%|█████████████████████████████████████████████████████████████████████████████| 42754/42754 [12:45<00:00, 55.82it/s]

2022-10-15 15:04:06,166 P56629 INFO Train loss: 0.552081
2022-10-15 15:04:06,167 P56629 INFO ************ Epoch=1 end ************


...

2022-10-15 16:11:17,778 P56629 INFO Train loss: 0.421477
2022-10-15 16:11:17,778 P56629 INFO ************ Epoch=2 end ************



100%|████████████████████████████████████████████████████████████████████████████| 10689/10689 [00:11<00:00, 920.78it/s]

2022-10-15 16:37:00,790 P56629 INFO [Metrics] logloss: 0.634303 - AUC: 0.710802
2022-10-15 16:37:00,791 P56629 INFO Monitor(max) STOP: 0.710802 !
2022-10-15 16:37:00,792 P56629 INFO Reduce learning rate on plateau: 0.000010
2022-10-15 16:37:00,792 P56629 INFO Early stopping at epoch=3
2022-10-15 16:37:00,793 P56629 INFO --- 42754/42754 batches finished ---



100%|████████████████████████████████████████████████████████████████████████████▉| 42753/42754 [25:48<00:00, 27.62it/s]

2022-10-15 16:37:05,798 P56629 INFO Train loss: 0.362447
2022-10-15 16:37:05,798 P56629 INFO Training finished.


In [27]:
model.load_weights(model.checkpoint) # reload the best checkpoint
logging.info('***** validation results *****')
model.evaluate_generator(valid_gen)

# logging.info('***** validation results *****')
# test_gen = datasets.h5_generator(feature_map, 
#                                  stage='test',
#                                  test_data=os.path.join(data_dir, 'test.h5'),
#                                  batch_size=params['batch_size'],
#                                  shuffle=False)
# model.evaluate_generator(test_gen)

2022-10-15 16:53:51,874 P56629 INFO ***** validation results *****


100%|████████████████████████████████████████████████████████████████████████████| 10689/10689 [00:11<00:00, 939.72it/s]

2022-10-15 16:54:03,367 P56629 INFO [Metrics] logloss: 0.536154 - AUC: 0.739063


{'logloss': 0.5361543050809597, 'AUC': 0.739063152050163}

In [14]:
from lightfm import LightFM
from lightfm.datasets import fetch_movielens
from lightfm.evaluation import precision_at_k

# Load the MovieLens 100k dataset. Only five
# star ratings are treated as positive.
data = fetch_movielens(min_rating=5.0)

ModuleNotFoundError: No module named 'lightfm'